# Evaluate & Visualize

This notebook computes the classification report, confusion matrix,
and saves a few overlay examples per class.


In [ ]:
import os, json, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import cv2

CLASSES = ['glioma','meningioma','no_tumor','pituitary_tumor']

# Example: load predictions/targets from CSV if you have it
# Expected columns: image_path, y_true (class string), y_pred (class string)
csv_path = os.environ.get('EVAL_CSV', 'predictions.csv')
if not os.path.exists(csv_path):
    # Build a small demo dataframe if CSV not present
    data = {
        'image_path': [],
        'y_true': [],
        'y_pred': []
    }
    df = pd.DataFrame(data)
else:
    df = pd.read_csv(csv_path)
df.head()

In [ ]:
# If you also have per-class probabilities, place them in y_prob_* columns
has_probs = all([f'y_prob_{c}' in df.columns for c in CLASSES])
y_true = df['y_true'].values if 'y_true' in df.columns else []
y_pred = df['y_pred'].values if 'y_pred' in df.columns else []

if len(y_true) and len(y_pred):
    print(classification_report(y_true, y_pred, labels=CLASSES, digits=4))
    cm = confusion_matrix(y_true, y_pred, labels=CLASSES)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASSES)
    plt.figure()
    disp.plot(values_format='d')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print('No data rows found (predictions.csv missing). Add your CSV to compute metrics.')

In [ ]:
# Save 3–5 examples per class (requires image_path column and optional segmentation masks)
OUT_DIR = 'figures_samples'
os.makedirs(OUT_DIR, exist_ok=True)

def save_example(img_path, out_path):
    img = cv2.imread(img_path)
    if img is None:
        return
    cv2.imwrite(out_path, img)

if len(y_true):
    for c in CLASSES:
        sub = df[df['y_true']==c].head(3)  # take first 3
        for i, row in sub.iterrows():
            ip = row['image_path']
            op = os.path.join(OUT_DIR, f'{c}_example_{i}.png')
            save_example(ip, op)
    print(f'Saved examples to: {OUT_DIR}')
else:
    print('Add predictions.csv with image_path to export sample figures.')